<a href="https://colab.research.google.com/github/jagatabhay/S5/blob/master/Copy_of_Copy_of_EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

**Target**: create an architecture with less than 10k parameters so accordingly chooses the imput and output channels.Here I have choosen the channels between 10 and 20.

**Analysis**: Adding dropout after max pooling has yielded better results in terms
of Accuracy rather than after each layer

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        #self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        #self.pool1 = nn.MaxPool2d(2, 2)
        #self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        #self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        #self.pool2 = nn.MaxPool2d(2, 2)
        #self.conv5 = nn.Conv2d(256, 512, 3)
        #self.conv6 = nn.Conv2d(512, 1024, 3)
        #self.conv7 = nn.Conv2d(1024, 10, 3)
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1,out_channels=10,kernel_size=(3,3),padding=0),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(num_features=10),
                                   #nn.Dropout(0.1)
                                   )

        self.conv2 = nn.Sequential(nn.Conv2d(in_channels=10,out_channels=16,kernel_size=(3,3),padding=0),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(num_features=16),
                                   #nn.Dropout(0.1)
        )
        self.conv3 = nn.Sequential(nn.Conv2d(in_channels=16,out_channels=20,kernel_size=(3,3),padding=0),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(num_features=20),
                                   #nn.Dropout(0.1)
                                   )                          
        
        self.pool1 =  nn.MaxPool2d(2,2)
         
        
        
        self.conv4 = nn.Sequential(nn.Conv2d(in_channels=20,out_channels=10,kernel_size=(1,1),padding=0),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(num_features=10),
                                   nn.Dropout(0.1)
                                   ) 
        
        #self.pool2 =  nn.MaxPool2d(2,2)

        self.conv5 = nn.Sequential(nn.Conv2d(in_channels=10,out_channels=20,kernel_size=(3,3),padding=0),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(num_features=20),
                                   #nn.Dropout(0.1)
                                   ) 
        
        self.conv6 = nn.Sequential(nn.Conv2d(in_channels=20,out_channels=16,kernel_size=(3,3),padding=0),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(num_features=16),
                                   #nn.Dropout(0.1)
                                   ) 
        self.conv7 = nn.Sequential(nn.Conv2d(in_channels=16,out_channels=10,kernel_size=(1,1),padding=0)
                                   #nn.ReLU()
                                   #nn.BatchNorm2d(num_features=10)
                                   #nn.DroupOut2d()
                                   )
        self.gap  =  nn.Sequential(nn.AvgPool2d(kernel_size=7)) 

        
        
        
        self.dropout = nn.Dropout(0.1)

              
        



    def forward(self, x):
        #x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        #x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))
        #x = F.relu(self.conv7(x))
        #x = x.view(-1, 10)
        x = self.conv1(x)
        x = self.conv2(x)
        
        x = self.conv3(x)
        x = self.pool1(x)
        x = self.dropout(x)
        x = self.conv4(x)
        #x = self.pool2(x)
        x = self.conv5(x)
        x = self.conv6(x)
        
        x = self.conv7(x)
        x = self.gap(x)
        

        x = x.view(-1, 10)
        



        return F.log_softmax(x)

Summary of our model architecture.

1. Used Batch Normalization at every layer and dropout after max pooling layer

2. Used GAP of 7*7 at the end

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
              ReLU-2           [-1, 10, 26, 26]               0
       BatchNorm2d-3           [-1, 10, 26, 26]              20
            Conv2d-4           [-1, 16, 24, 24]           1,456
              ReLU-5           [-1, 16, 24, 24]               0
       BatchNorm2d-6           [-1, 16, 24, 24]              32
            Conv2d-7           [-1, 20, 22, 22]           2,900
              ReLU-8           [-1, 20, 22, 22]               0
       BatchNorm2d-9           [-1, 20, 22, 22]              40
        MaxPool2d-10           [-1, 20, 11, 11]               0
          Dropout-11           [-1, 20, 11, 11]               0
           Conv2d-12           [-1, 10, 11, 11]             210
             ReLU-13           [-1, 10, 11, 11]               0
      BatchNorm2d-14           [-1, 10,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:96: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Apply RamdomRotation to image by 7 degree for getting better accuracy tried 

with other degrees like 5,6 and 10 this was best . 

 converting the image to tensor and normalizing the pixel of image.Used Random Affine image augmentation techniques.

We are doing image Augmentation on the train dataset not on testing dataset.
Used ColorJitter with different combination of parameters as well as resize function of transform but didnot gave better results in terms of accuracy so commented it.

In [0]:


torch.manual_seed(1)
batch_size = 30

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        #transforms.Resize((30,30)),                          
                        transforms.RandomRotation((-7.0,7.0),fill=(1,)),
                        transforms.RandomAffine(degrees=10, translate=(0.1,0.1), scale=(0.9, 1.1)),
                        #transforms.ColorJitter(brightness=0.1, contrast=0.1,saturation=0.1,hue=0.1),                          
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
train_losses=[]
#test_losses=[]
train_acc=[]
test_acc=[]
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        y_pred = model(data)
        loss = F.nll_loss(y_pred, target)
        train_losses.append(loss)
        loss.backward()
        optimizer.step()

        pred=y_pred.argmax(dim=1,keepdim=True)
        correct+=pred.eq(target.view_as(pred)).sum().item()
        processed+=len(data)
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Using epoch of 15.Using StepLR function from lr schedular module using step_size of 5 is giving better accuracy compared to 4 and 6.This is basically done by Hit and Trial.

In [0]:
from torch.optim.lr_scheduler import StepLR
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler= StepLR(optimizer,step_size=5,gamma=0.1)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/2000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:96: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.1441677063703537 batch_id=1999 Accuracy=88.55: 100%|██████████| 2000/2000 [00:47<00:00, 41.94it/s]
  0%|          | 0/2000 [00:00<?, ?it/s]


Test set: Average loss: 0.0658, Accuracy: 9798/10000 (97.98%)



loss=0.04076778143644333 batch_id=1999 Accuracy=95.92: 100%|██████████| 2000/2000 [00:47<00:00, 42.00it/s]
  0%|          | 0/2000 [00:00<?, ?it/s]


Test set: Average loss: 0.0417, Accuracy: 9865/10000 (98.65%)



loss=0.14718104898929596 batch_id=1999 Accuracy=96.73: 100%|██████████| 2000/2000 [00:47<00:00, 41.74it/s]
  0%|          | 0/2000 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0179, Accuracy: 9943/10000 (99.43%)



loss=0.01386117935180664 batch_id=1999 Accuracy=97.98: 100%|██████████| 2000/2000 [00:47<00:00, 42.06it/s]
  0%|          | 0/2000 [00:00<?, ?it/s]


Test set: Average loss: 0.0188, Accuracy: 9934/10000 (99.34%)



loss=0.031740952283144 batch_id=1999 Accuracy=98.09: 100%|██████████| 2000/2000 [00:47<00:00, 41.79it/s]



Test set: Average loss: 0.0177, Accuracy: 9940/10000 (99.40%)



**Target:**
1. Getting an accuracy of 99.4 with less than 10k parametrs under 15 epochs.
Results:
1. Parameters: 90736 
2. Best Train Accuracy:98.09
3. Best Test Accuracy: 99.45

**Analysis:**

1. The model is doing pretty well not overfitting as well as getting consistent accuracy.

2. Seeing image sample ,we get to know we can add slight rotation for better accuracy.
3. Doing image augmentation and choosing the step size of LR schedular wisely can improve the accuracy.

4. Adding too many image augmentation techniques work in a negative way reduces the accuracy or have not much impact